# TAREA 3
---

In [90]:
# Importamos 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from pandasql import sqldf
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score

In [91]:
# Eliminamos los warnings
warnings.filterwarnings("ignore")

In [92]:
# Cargamos el dataset y visualizamos las 3 primeras muestras
df_cars = pd.read_csv("./vw.csv")
df_cars.head(3)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,T-Roc,2019,25000,Automatic,13904,Diesel,145,49.6,2.0
1,T-Roc,2019,26883,Automatic,4562,Diesel,145,49.6,2.0
2,T-Roc,2019,20000,Manual,7414,Diesel,145,50.4,2.0


In [93]:
# Definimos la función para los "queries"
pysqldf = lambda q: sqldf(q, globals())

In [94]:
# Hacemos una llamada de query y seleccionamos solamente las columnas model, transmission y furlType
query = """SELECT model, transmission, fuelType FROM df_cars"""
dfCars = pysqldf(query)
dfCars.head(3)

,model,transmission,fuelType
0,T-Roc,Automatic,Diesel
1,T-Roc,Automatic,Diesel
2,T-Roc,Manual,Diesel


In [95]:
# Procedemos a codificar las model, transmission y furlType
dfCars["modelEncode"] = dfCars.model.astype("category").cat.codes
dfCars["transmissionEncode"] = dfCars.transmission.astype("category").cat.codes
dfCars["fuelTypeEncode"] = dfCars.fuelType.astype("category").cat.codes

In [96]:
# Visualizamos 3 muestras aleatorias
dfCars.sample(3)

,model,transmission,fuelType,modelEncode,transmissionEncode,fuelTypeEncode
14526,Golf SV,Semi-Auto,Petrol,13,2,3
10729,Tiguan,Semi-Auto,Petrol,22,2,3
7366,Polo,Manual,Petrol,16,1,3


In [97]:
# Nuestras variables predictoras serán transmissionEncode y fuelTypeEncode
X = dfCars[["transmissionEncode", "fuelTypeEncode"]]
# Nuestra variable objetivo será modelEncode
y = dfCars["modelEncode"]

In [98]:
# Hacemos la partición de los datos de entranmiento y de test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

In [99]:
# Validación cruzada del modelo de árbol de decisón
param = {"max_depth" : [2, 4, 6]}
dtc_model = DecisionTreeClassifier(random_state = 42)
grid_search = GridSearchCV(dtc_model, param)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [2, 4, 6]})

In [100]:
# Mejor resultado del modelo de árbol de decisón
grid_search.best_params_

{'max_depth': 4}

In [101]:
# Validación cruzada del modelo de K-Vcinos
param = {"n_neighbors" : [5, 10, 15]}
knc_model = KNeighborsClassifier()
grid_search = GridSearchCV(knc_model, param)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 10, 15]})

In [102]:
# Mejor resultado del modelo de K-Vcinos
grid_search.best_params_

{'n_neighbors': 15}

In [103]:
# Validación cruzada del modelo de Máquina de Vectores de Soporte
param = {"kernel" : ["linear", "poly", "rbf"]}
svc_model = SVC()
grid_search = GridSearchCV(svc_model, param)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=SVC(), param_grid={'kernel': ['linear', 'poly', 'rbf']})

In [104]:
# Mejor resultado del modelo de Máquina de Vectores de Soporte
grid_search.best_params_

{'kernel': 'rbf'}

In [105]:
# Instanciamos el modelo del Árbol de Decisión
dtc_model2 = DecisionTreeClassifier(max_depth = 4, random_state = 42)
# Entrenemos el modelo
dtc_model2.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4, random_state=42)

In [106]:
dtc_model2.score(X_train, y_train)

0.3917525773195876

In [107]:
# Instanciamos el modelo del K-Vecinos
knc_model2 = KNeighborsClassifier(n_neighbors = 15)
# Entrenemos el modelo
knc_model2.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=15)

In [108]:
# Preparamos los estimadores para el modelo de Stacking Classifier
estimadores = [
    ("cart", DecisionTreeClassifier(max_depth = 4, random_state = 0)),
    ("knn", KNeighborsClassifier(n_neighbors = 15))
]

# Instanciamos el modelo del Stacking Classifier
stacking_clf = StackingClassifier(estimators = estimadores, final_estimator = SVC(kernel = "rbf"))
# Entrenamos el modelo
stacking_clf.fit(X_train, y_train)
score_stck = stacking_clf.score(X_test, y_test)
print(score_stck)

0.39907651715039577


In [ ]:
# Diccionario con los modelos
mdls = {
    "Árbol de Decisión": dtc_model,
    "K-Nearest Neighbors": knc_model,
    "Stacking": stacking_clf
}

# Instanciamos diccionario "resultados" vacío 
resultados = {}

# Bucle que recorrerá los modelos, imprimiendo el nombre de cada uno de ellos y obtendrá los scores de cada modelo
for mdl_name, mdl in mdls.items():
    print(f"Comienzo de entrenamiento del modelo {mdl_name}")
    mdl.fit(X_train, y_train)
    y_predct = mdl.predict(X_test)
    score = accuracy_score(y_test, y_predct)
    resultados[mdl_name] = score
    
# Bucle que imprimirá los scores de cada modelo
for mdl_name, score in resultados.items():
    print(f"Score del modelo {mdl_name}: {score}")
    
# Dibujamos la gráfica de barras
plt.bar(range(len(resultados)), list(resultados.values()))
plt.xticks(range(len(resultados)), list(resultados.keys()))
plt.title("Gráfica de comparación de los scores de los modelos")
plt.xlabel("Modelos")
plt.ylabel("Score")
plt.show()

Comienzo de entrenamiento del modelo Árbol de Decisión
Comienzo de entrenamiento del modelo K-Nearest Neighbors
Comienzo de entrenamiento del modelo Stacking


### Observación:

En este caso **los modelos que mejor rendimiento tienen son el Arbol de Decisión y el Stacking** los dos con un score de: **0.39907651715039577**